<a href="https://colab.research.google.com/github/AmeerHamza-345/Q2/blob/main/Project_2_LangChain_RAG_Project(Smart_Shopping_Assistant).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --q --upgrade langchain pinecone-client tqdm
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [2]:
import langchain_google_genai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
PINECONE_API_KEY= userdata.get('PINECONE_API_KEY')
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
model_2_0:ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
  model = "gemini-2.0-flash-exp",
  api_key = GOOGLE_API_KEY
)

In [5]:
response = model_2_0.invoke("hello")
print(response.content)

Hello! How can I help you today?


In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/smart_shopping_assistant.csv")

dataset = loader.load()

print(dataset)

[Document(metadata={'source': '/content/smart_shopping_assistant.csv', 'row': 0}, page_content='Product Name: Product 1\nCategory: Electronics\nPrice: $60\nAvailability: In Stock\nDescription: Description for Product 1'), Document(metadata={'source': '/content/smart_shopping_assistant.csv', 'row': 1}, page_content='Product Name: Product 2\nCategory: Clothing\nPrice: $70\nAvailability: Out of Stock\nDescription: Description for Product 2'), Document(metadata={'source': '/content/smart_shopping_assistant.csv', 'row': 2}, page_content='Product Name: Product 3\nCategory: Home Appliances\nPrice: $80\nAvailability: In Stock\nDescription: Description for Product 3'), Document(metadata={'source': '/content/smart_shopping_assistant.csv', 'row': 3}, page_content='Product Name: Product 4\nCategory: Books\nPrice: $90\nAvailability: Out of Stock\nDescription: Description for Product 4'), Document(metadata={'source': '/content/smart_shopping_assistant.csv', 'row': 4}, page_content='Product Name: Pro

In [9]:
from langchain.text_splitter import CharacterTextSplitter # Import the CharacterTextSplitter class
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(dataset)

In [10]:
print(chunks[0])
print(len(chunks))

page_content='Product Name: Product 1
Category: Electronics
Price: $60
Availability: In Stock
Description: Description for Product 1' metadata={'source': '/content/smart_shopping_assistant.csv', 'row': 0}
1000


In [11]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
# os.environ['PINECONE_ENV'] = userdata.get('pinecone_env')
api_key = os.environ.get('PINECONE_API_KEY')

In [12]:
PINECONE_API_KEY=PINECONE_API_KEY
PINECONE_ENVIRONMENT="us-east-1"
GOOGLE_API_KEY=GOOGLE_API_KEY

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#create embedding means numeric representation of text provided or document
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # lates model for embedding from google
em=embeddings.embed_query("H")
em[:3]

[0.0237551461905241, -0.060975316911935806, -0.0640849843621254]

In [14]:
from langchain_community.vectorstores import VectorStore
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

In [15]:
index_name = "shopping-assistant" # name must be lower case and ''
pc = Pinecone(api_key=api_key)

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)
print(index)
vector_store = PineconeVectorStore(embedding=embeddings, index=index) #Vector Store

In [16]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
# vectors=[]
for data in tqdm(dataset):
    vector = embeddings.embed_query(data.page_content) #create embedding numeric

    # Generate a unique ID for each document (e.g., UUID or hash)
    data_id = str(hash(data.page_content)) #uuid or Hash doc_id = 22i3y12jdyuew****

    # Create metadata dictionary
    data_metadata = {"source": data.metadata["source"]}
    data_list=[data]
    # Upsert the vector with a unique ID and metadata
    # index.upsert(vectors=[(doc_id, vector, doc_metadata)]) # Include metadata in upsert
    vector_store.add_documents(documents=data_list, id=data_id)

 89%|████████▉ | 889/1000 [08:35<01:04,  1.72it/s]


GoogleGenerativeAIError: Error embedding content: 429 Quota exceeded for quota metric 'Batch Embed Content API requests' and limit 'Batch embed contents request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:502430975926'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/batch_embed_contents_requests"
}
metadata {
  key: "quota_location"
  value: "us-west4"
}
metadata {
  key: "quota_limit"
  value: "BatchEmbedContentsRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "150"
}
metadata {
  key: "consumer"
  value: "projects/502430975926"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quotas/help/request_increase"
}
]

In [17]:
retriver = vector_store.as_retriever(search_kwargs={"k": 1})
#print(retriver)


In [28]:
from pprint import pprint
user_quory:str = "show five bags "
docs = retriver.get_relevant_documents(user_quory)
pprint(docs)

[Document(id='2111d18c-ec80-477e-b1e6-519dbde0db24', metadata={'row': 49.0, 'source': '/content/smart_shopping_assistant.csv'}, page_content='Product Name: Product 50\nCategory: Toys\nPrice: $550\nAvailability: Out of Stock\nDescription: Description for Product 50')]


In [29]:
prompt_template = f"""Give answer of {user_quory} from the rag context {docs} """
response = model_2_0.invoke(prompt_template)

In [30]:
pprint(response.content)

("Okay, I understand you're asking me to list five bags based on the provided "
 "context. However, the context you've given me describes a single product, "
 '"Product 50", which is a **toy** and not a bag. It also indicates it is "Out '
 'of Stock". \n'
 '\n'
 'Therefore, I cannot list five bags from this context. \n'
 '\n'
 "**It seems like the context you provided doesn't contain any information "
 'about bags.**\n'
 '\n'
 'To help me give you a better answer, please provide a context that includes '
 'information about bags. For example, you could provide a list of products '
 'with their categories and descriptions, or a document that specifically '
 'talks about bags.')
